In [12]:
## Get data
import pandas as pd

fetal_health = pd.read_csv('fetal_health.csv')

fetal_health.histogram_tendency = fetal_health.histogram_tendency.astype('int').astype('str')

fetal_health = pd.get_dummies(fetal_health) ## I need to figure out how to get this to work when only passing one obs

fetal_health.fetal_health = fetal_health.fetal_health.astype('int').astype('str')

In [13]:
from sklearn.model_selection import train_test_split

fetal_health, fetal_health_test = train_test_split(fetal_health, 
                                    test_size = .2) ## withold our test set

fetal_health_train, fetal_health_validate = train_test_split(fetal_health, 
                                    test_size = .2) 



In [14]:
from utility_functions import rebalance_classes

fetal_health_train = rebalance_classes(fetal_health_train)

In [15]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

fetal_health_train_scaler = scale.fit(X = fetal_health_train.drop('fetal_health', axis = 1).to_numpy())

fetal_health_train_x = fetal_health_train_scaler.transform(\
    X = fetal_health_train.drop('fetal_health', axis = 1).to_numpy())
                                   

fetal_health_validate_x = fetal_health_train_scaler.transform(\
    X = fetal_health_validate.drop('fetal_health', axis = 1).to_numpy())


    



In [16]:
import tensorflow as tf
import numpy as np


train_y = pd.get_dummies(fetal_health_train.fetal_health).to_numpy()
validate_y = pd.get_dummies(fetal_health_validate.fetal_health).to_numpy()

train_tf_dataset = tf.data.Dataset.from_tensor_slices((fetal_health_train_x, train_y))
validate_tf_dataset = tf.data.Dataset.from_tensor_slices((fetal_health_validate_x, validate_y))


In [17]:
from tensorflow import keras
from tensorflow.keras import layers


BATCH_SIZE = 10

train_tf_dataset = train_tf_dataset.shuffle(buffer_size = fetal_health_train_x.shape[0]).batch(BATCH_SIZE)

validate_tf_dataset = validate_tf_dataset.batch(BATCH_SIZE)


In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(23, input_dim=23, activation = 'relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation = 'softmax')
])


model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss='categorical_crossentropy') #use because classes are balanced

In [19]:
model.fit(train_tf_dataset, 
          validation_data = validate_tf_dataset,
          epochs=20)

Epoch 1/20
317/317 [==============================] - 4s 13ms/step - loss: 0.5001 - val_loss: 0.0000e+00
Epoch 2/20
317/317 [==============================] - 1s 5ms/step - loss: 0.3140 - val_loss: 0.3207
Epoch 3/20
317/317 [==============================] - 1s 3ms/step - loss: 0.2504 - val_loss: 0.2886
Epoch 4/20
317/317 [==============================] - 1s 4ms/step - loss: 0.2071 - val_loss: 0.2861
Epoch 5/20
317/317 [==============================] - 1s 4ms/step - loss: 0.1797 - val_loss: 0.3179
Epoch 6/20
317/317 [==============================] - 3s 10ms/step - loss: 0.1535 - val_loss: 0.2502
Epoch 7/20
317/317 [==============================] - 2s 5ms/step - loss: 0.1361 - val_loss: 0.3063
Epoch 8/20
317/317 [==============================] - 1s 4ms/step - loss: 0.1222 - val_loss: 0.2989
Epoch 9/20
317/317 [==============================] - 1s 5ms/step - loss: 0.1098 - val_loss: 0.3069
Epoch 10/20
317/317 [==============================] - 1s 3ms/step - loss: 0.0968 - val_loss: 

In [12]:
## Looks like our previous model was overfitting after 2 epochs, so let's see if we can reduce that a bit. 

3186

(1700, 21)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,-0.833536,2.833079,-0.178197,-1.110651,-0.608517,-0.114396,-0.456346,-1.336023,0.021985,-0.745624,...,1.204357,-1.055394,0.966302,0.734261,-0.443921,-0.023776,0.376939,-0.002029,0.040500,-0.310440
1,-0.129078,-0.517716,-0.229352,0.447382,0.909666,-0.114396,-0.456346,-1.394128,0.225727,0.009529,...,1.001679,-1.323008,-0.000711,-0.262977,1.114713,0.109241,-0.094123,0.050765,0.063792,1.203134
2,-0.531625,-0.517716,5.670473,-1.110651,-0.608517,-0.114396,2.413750,1.336828,-0.894852,-0.745624,...,-0.372022,-0.044406,-0.967724,-0.262977,-0.443921,-1.132251,-0.376761,-0.688357,0.343296,1.203134
3,0.676017,3.838317,0.623224,-0.487438,-0.608517,-0.114396,-0.456346,-1.103601,-0.079885,-0.745624,...,0.236010,0.431353,1.342363,-0.595389,-0.443921,1.882801,1.978549,2.162543,-0.308879,1.203134
4,1.279838,1.492761,-0.229352,1.693809,-0.608517,-0.114396,-0.456346,-2.091394,0.633210,-0.745624,...,-0.214384,0.936847,1.181194,-0.595389,-0.443921,1.040360,1.130638,1.106654,-0.402047,-0.310440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,0.776654,-0.517716,-0.229352,-0.799044,-0.608517,-0.114396,-0.456346,0.697668,-0.894852,-0.519078,...,-0.867455,1.026052,-0.215603,-0.927801,-0.443921,0.508292,0.612470,0.578710,-0.541798,-0.310440
336,-0.129078,0.487523,-0.109991,-0.799044,-0.608517,-0.114396,-0.456346,-0.232019,2.568756,-0.745624,...,1.452073,-1.293273,1.127471,-0.927801,-0.443921,0.109241,0.188514,0.209149,0.017208,-0.310440
337,-1.236083,2.833079,-0.212301,-0.175831,-0.608517,-0.114396,-0.456346,-0.115808,0.836952,-0.745624,...,0.213490,-0.371490,-0.161880,0.734261,-0.443921,0.286597,0.141408,0.156354,-0.332171,1.203134
338,-0.229715,-0.517716,-0.212301,0.447382,0.302393,-0.114396,1.457051,0.232825,1.142564,-0.745624,...,0.889081,-1.114863,0.106735,0.069436,-0.443921,-0.245471,-1.318884,-1.005124,1.065346,-0.310440


25

1700